# Retrievers
- https://python.langchain.com/docs/integrations/retrievers/

## ArxivRetriever
- https://python.langchain.com/docs/integrations/retrievers/arxiv/

In [1]:
!uv pip install -qU langchain-community arxiv

In [2]:
from langchain_community.retrievers import ArxivRetriever

retriever = ArxivRetriever(
    load_max_docs=2,
    get_ful_documents=True,
)

In [3]:
docs = retriever.invoke("1605.08386")
docs[0].metadata  # meta-information of the Document

{'Entry ID': 'http://arxiv.org/abs/1605.08386v1',
 'Published': datetime.date(2016, 5, 26),
 'Title': 'Heat-bath random walks with Markov bases',
 'Authors': 'Caprice Stanley, Tobias Windisch'}

In [4]:
docs[0].page_content[:400]  # a content of the Document

'Graphs on lattice points are studied whose edges come from a finite set of\nallowed moves of arbitrary length. We show that the diameter of these graphs on\nfibers of a fixed integer matrix can be bounded from above by a constant. We\nthen study the mixing behaviour of heat-bath random walks on these graphs. We\nalso state explicit conditions on the set of moves so that the heat-bath random\nwalk, a ge'

In [5]:
docs = retriever.invoke("What is the ImageBind model?")
docs[1].metadata  # meta-information of the Document

{'Entry ID': 'http://arxiv.org/abs/2309.03905v2',
 'Published': datetime.date(2023, 9, 11),
 'Title': 'ImageBind-LLM: Multi-modality Instruction Tuning',
 'Authors': 'Jiaming Han, Renrui Zhang, Wenqi Shao, Peng Gao, Peng Xu, Han Xiao, Kaipeng Zhang, Chris Liu, Song Wen, Ziyu Guo, Xudong Lu, Shuai Ren, Yafei Wen, Xiaoxin Chen, Xiangyu Yue, Hongsheng Li, Yu Qiao'}

In [6]:
docs[1].page_content[:400]  # a content of the Document

'We present ImageBind-LLM, a multi-modality instruction tuning method of large\nlanguage models (LLMs) via ImageBind. Existing works mainly focus on language\nand image instruction tuning, different from which, our ImageBind-LLM can\nrespond to multi-modality conditions, including audio, 3D point clouds, video,\nand their embedding-space arithmetic by only image-text alignment training.\nDuring training'

In [7]:
# 顯示每篇文件的 metadata
for i, doc in enumerate(docs):
    print(f"\n[Doc {i+1}] Metadata:")
    for key, value in doc.metadata.items():
        print(f"  {key}: {value}")


[Doc 1] Metadata:
  Entry ID: http://arxiv.org/abs/2305.05665v2
  Published: 2023-05-31
  Title: ImageBind: One Embedding Space To Bind Them All
  Authors: Rohit Girdhar, Alaaeldin El-Nouby, Zhuang Liu, Mannat Singh, Kalyan Vasudev Alwala, Armand Joulin, Ishan Misra

[Doc 2] Metadata:
  Entry ID: http://arxiv.org/abs/2309.03905v2
  Published: 2023-09-11
  Title: ImageBind-LLM: Multi-modality Instruction Tuning
  Authors: Jiaming Han, Renrui Zhang, Wenqi Shao, Peng Gao, Peng Xu, Han Xiao, Kaipeng Zhang, Chris Liu, Song Wen, Ziyu Guo, Xudong Lu, Shuai Ren, Yafei Wen, Xiaoxin Chen, Xiangyu Yue, Hongsheng Li, Yu Qiao

[Doc 3] Metadata:
  Entry ID: http://arxiv.org/abs/2503.03648v1
  Published: 2025-03-05
  Title: Modelowanie nieliniowej charakterystyki szerokopasmowych wzmacniaczy radiowych o zmiennym napięciu zasilania; Modeling Nonlinear Characteristics of Wideband Radio Frequency Amplifiers with Variable Supply Voltage
  Authors: Kornelia Kostrzewska, Paweł Kryszkiewicz


# chain

In [8]:
# chain
# https://build.nvidia.com/deepseek-ai/deepseek-r1
# nvapi-xxx
import getpass
import os
if not os.environ.get("NVIDIA_API_KEY"):
  os.environ["NVIDIA_API_KEY"] = getpass.getpass("Enter API key for NVIDIA: ")

from langchain.chat_models import init_chat_model
llm = init_chat_model("meta/llama-4-maverick-17b-128e-instruct", model_provider="nvidia")

Enter API key for NVIDIA:  ········


D:\uv\langchaing\Lib\site-packages\langchain_nvidia_ai_endpoints\_common.py:212: UserWarning: Found meta/llama-4-maverick-17b-128e-instruct in available_models, but type is unknown and inference may fail.
  warnings.warn(


In [9]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.retrievers import ArxivRetriever


retriever = ArxivRetriever(load_max_docs=2, get_ful_documents=True)

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)
query = "What is the ImageBind model?"
chain.invoke({"input": query})


{'input': 'What is the ImageBind model?',
 'context': [Document(metadata={'Entry ID': 'http://arxiv.org/abs/2305.05665v2', 'Published': datetime.date(2023, 5, 31), 'Title': 'ImageBind: One Embedding Space To Bind Them All', 'Authors': 'Rohit Girdhar, Alaaeldin El-Nouby, Zhuang Liu, Mannat Singh, Kalyan Vasudev Alwala, Armand Joulin, Ishan Misra'}, page_content="We present ImageBind, an approach to learn a joint embedding across six\ndifferent modalities - images, text, audio, depth, thermal, and IMU data. We\nshow that all combinations of paired data are not necessary to train such a\njoint embedding, and only image-paired data is sufficient to bind the\nmodalities together. ImageBind can leverage recent large scale vision-language\nmodels, and extends their zero-shot capabilities to new modalities just by\nusing their natural pairing with images. It enables novel emergent applications\n'out-of-the-box' including cross-modal retrieval, composing modalities with\narithmetic, cross-modal

##   作業

**擴展檢索鏈 (Retrieval Chain)**

Notebook 中創建了一個基本的檢索鏈，將 ArxivRetriever 和 LLM 連接起來回答問題。

1.  **基本功能：**
    * 在 notebook 的基礎上，擴展檢索鏈的功能，使其更加實用。

2.  **擴展方向（選擇至少兩項）：**
    * **文件篩選：** 讓使用者可以根據作者、日期、關鍵字等條件篩選文獻。
    * **摘要生成：** 自動總結檢索到的文獻內容。
    * **多源檢索：** 整合多個 Retriever，從不同來源檢索資訊 (例如 Arxiv、Web、自訂資料庫)。
    * **結果排序：** 根據相關性對檢索結果進行排序和評分。
    * **複雜查詢處理：** 處理包含多個關鍵字和條件的查詢。

**評估標準：**

* 檢索鏈是否能夠正確檢索並呈現相關文獻？ (50%)
* 擴展功能是否能夠有效提升檢索的實用性？ (50%)

